### Notebook Status

Goal 1 - General a url list
Used earthaccess to generate urls, but only https urls, not s3. S3 would be better because I think with that I could use xarray to access each file and do the subsetting.

Goal 2 - Open an s3 url
I am not able to use the workflow I was using with IS2 to open the data. The last snippet of xarray code runs fine in Cryocloud, but not from this notebook run locally. Hmm.

1 - Get the MUR urls

In [ ]:
import earthaccess

This takes me: ~ 4 mintutes

In [ ]:
%%time

result = earthaccess.search_data(
    short_name='MUR-JPL-L4-GLOB-v4.1',
    temporal=('2002-09-01', '2023-09-01'),
    cloud_hosted=True,
    count = 10,  # uncomment this to just download the first 50 files
)

In [ ]:
result

In [ ]:
result[0]._derive_s3_link(result[0].data_links())

In [ ]:
import xarray as xr

In [ ]:
auth = earthaccess.login()

In [ ]:
url = 's3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20230917090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'


In [ ]:
xr.open_dataset(
    's3://podaac-ops-cumulus-protected/MUR-JPL-L4-GLOB-v4.1/20230917090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
)

### EA Note

I really don't think this should fail silently. It makes debugging really hard.

In [ ]:
s3_creds = auth.get_s3_credentials(daac='NSIDC')
print(s3_creds.keys())

In [ ]:
s3 = earthaccess.get_s3fs_session(daac="PODAAC", provider=auth)

In [ ]:
from s3fs import S3FileSystem

In [ ]:
s3 = S3FileSystem(
        key=s3_creds['accessKeyId'], 
        secret=s3_creds['secretAccessKey'], 
        token=s3_creds['sessionToken']
)  

In [ ]:
url = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL03/006/2019/11/30/ATL03_20191130112041_09860505_006_01.h5'

In [ ]:
url

In [ ]:
with s3.open(url, 'rb'):
    print('hi')

### Confirming it will work with is2

This code works on cryocloud, but not locally.

In [1]:
import earthaccess
import xarray as xr

In [2]:
earthaccess.__version__

'0.5.3'

In [3]:
auth = earthaccess.login()
s3 = earthaccess.get_s3fs_session(daac="NSIDC", provider=auth)

EARTHDATA_USERNAME and EARTHDATA_PASSWORD are not set in the current environment, try setting them or use a different strategy (netrc, interactive)
You're now authenticated with NASA Earthdata Login
Using token with expiration date: 10/13/2023
Using .netrc file for EDL


In [4]:
# Global Geolocated Photon Data
s3url_atl03 = 's3://nsidc-cumulus-prod-protected/ATLAS/ATL03/006/2019/11/30/ATL03_20191130112041_09860505_006_01.h5'

In [5]:
xr.open_dataset(s3.open(s3url_atl03, 'rb'))

PermissionError: Forbidden

In [ ]:
xr.open_dataset(s3.open(url, 'rb'))

In [ ]:
urls = [r.data_links() for r in result]

In [ ]:
len(urls)

In [ ]:
# I don't think I can do this because it won't subset in between
# downloaded_files = earthaccess.download(
#     results,
#     local_path='./data',
# )

In [ ]:
from earthaccess import DataGranules

In [ ]:
DataGranules().short_name('MUR-JPL-L4-GLOB-v4.1').hits()

In [ ]:
collection = DataGranules().concept_id('G2768603767-POCLOUD')

In [ ]:
collection.get()

2 - subset and download